In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import models

In [ ]:
path_data = "/content/drive/MyDrive/SuccKD_MultiSpecies/Data/"
path_model = "/content/drive/MyDrive/SuccKD_MultiSpecies/Model/"
path_result = "/content/drive/MyDrive/SuccKD_MultiSpecies/Result/"

In [ ]:
# Buildi an amino acid dictionary
def twoTupleDic1():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)


In [ ]:
# Load Train data
file_train = "Train_Generic_succ.csv"
df_train =pd.read_csv(path_data +file_train, delimiter= ',')

texts_train =[] #PTMsequend kmer
for i in df_train['Peptide_WS_33']:
  temp = ProSentence(i,k) # Biểu diễn dữ liệu đầu vào thành token Kmer
  texts_train.append(temp)
df_train['k_mer'] =texts_train
train_sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences.append(each_index_list)
# Tokenizer train data
data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())

MAX_SEQUENCE_LENGTH = len(data_token[1])

Xtrain= pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytrain = np.array(df_train['Label'])
Xtrain.shape
ytrain = np.array(ytrain)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
ytrain = lb.fit_transform(ytrain)
ytrain = to_categorical(ytrain)
ytrain.shape

(14250, 2)

In [ ]:
# load test data
file_test ="Test_Generic_succ.csv"
df_test =pd.read_csv(path_data+file_test,delimiter= ',')
text_test =[] #PTMsequend kmer
for i in df_test['Peptide_WS_33']:
  temp = ProSentence(i,k) # Biểu diễn dữ liệu đầu vào thành token Kmer
  text_test.append(temp)
df_test['k_mer'] =text_test

test_sequences = []
for each in text_test:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    test_sequences.append(each_index_list)

Xtest = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytest= np.array(df_test['Label'])
Xtest.shape
ytest = np.array(ytest)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
ytest= lb.fit_transform(ytest)
ytest = to_categorical(ytest)
ytest.shape

(3231, 2)

In [ ]:
TIME_STEPS = MAX_SEQUENCE_LENGTH
INPUT_SIZE =300

**KD2_Succi: 2teacher and 1 student**

In [ ]:
# 2 teacher and 1 student
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras import models, layers, losses

# Teacher 1
def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.4))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model

# Teacher 2
def create_bilstm_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
# Student
def create_student():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.2))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
# KD2_Succi: Knowlege Distiller Model
import tensorflow as tf
from tensorflow.keras import optimizers

class Distiller(tf.keras.Model):
    def __init__(self, teacher1, teacher2, student, alpha=0.5, temperature=3.0):
        super(Distiller, self).__init__()
        self.teacher1 = teacher1
        self.teacher2 = teacher2
        self.student = student
        self.alpha = alpha
        self.temperature = temperature

        # Optimizer for the student
        self.student_optimizer = AdamW(learning_rate=0.0001)

    def compile(self, metrics, student_loss_fn, distillation_loss_fn):
        super(Distiller, self).compile(metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn

    def call(self, inputs):
        """
        This method defines how the model processes inputs.
        Args:
        - inputs: Input data (Xtest_student or any other data).

        Returns:
        - Output logits from the student model.
        """
        student_logits = self.student(inputs, training=False)
        return student_logits

    def train_step(self, data):
        x, y = data

        # Sequentially forward pass through teachers
        teacher1_logits = self.teacher1(x, training=False)
        teacher2_logits = self.teacher2(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass through student
            student_logits = self.student(x, training=True)

            # Compute the task loss (for hard labels)
            student_loss = self.student_loss_fn(y, student_logits)

            # Compute distillation losses (for soft labels)
            distill_loss1 = self.distillation_loss_fn(
                tf.nn.softmax(teacher1_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            distill_loss2 = self.distillation_loss_fn(
                tf.nn.softmax(teacher2_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            # Combined loss
            total_loss = self.alpha * student_loss + (1 - self.alpha) * (distill_loss1 + distill_loss2)

        # Compute gradients and update student model
        grads = tape.gradient(total_loss, self.student.trainable_variables)
        self.student_optimizer.apply_gradients(zip(grads, self.student.trainable_variables))

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        # Return results
        results = {
            'student_loss': student_loss,
            'distill_loss1': distill_loss1,
            'distill_loss2': distill_loss2
        }
        results.update({m.name: m.result() for m in self.metrics})
        return results

    def test_step(self, data):
        x, y = data
        student_logits = self.student(x, training=False)

        # Compute the task loss (for hard labels)
        student_loss = self.student_loss_fn(y, student_logits)

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        return {
            'student_loss': student_loss,
            **{m.name: m.result() for m in self.metrics}
        }

# Instantiate models

teacher1 = create_cnn_model()
teacher2 = create_bilstm_model()
student = create_student()
teacher1.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[
        "categorical_accuracy",
        "AUC",
    ],
    optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
)
teacher2.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[
        "categorical_accuracy",
        "AUC",
    ],
    optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
)
teacher1.fit(
    Xtrain, ytrain,
    batch_size =16,
    epochs=50
)
teacher1.save(path_model +"model_teacher21.h5")

teacher2.fit(
    Xtrain, ytrain,
    batch_size =16,
    epochs=50
)
teacher2.save(path_model +"model_teacher22.h5")


Epoch 1/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - AUC: 0.6266 - categorical_accuracy: 0.5967 - loss: 0.7345
Epoch 2/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - AUC: 0.6666 - categorical_accuracy: 0.6459 - loss: 0.6628
Epoch 3/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - AUC: 0.6855 - categorical_accuracy: 0.6552 - loss: 0.6425
Epoch 4/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - AUC: 0.6945 - categorical_accuracy: 0.6606 - loss: 0.6353
Epoch 5/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - AUC: 0.7060 - categorical_accuracy: 0.6683 - loss: 0.6278
Epoch 6/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - AUC: 0.7216 - categorical_accuracy: 0.6725 - loss: 0.6161
Epoch 7/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - AUC: 0.7332 - categorical_accuracy: 0.6726 - loss: 0.6067
Epoch 8/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - AUC: 0.7299 - categorical_accuracy: 0.6746 - loss: 0.6089
Epoch 9/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - AUC: 0.7441 - categorical_accuracy: 0.6858 - loss:

Epoch 1/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 29s 26ms/step - AUC: 0.6317 - categorical_accuracy: 0.6004 - loss: 0.8517
Epoch 2/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 23s 26ms/step - AUC: 0.7456 - categorical_accuracy: 0.6816 - loss: 0.5960
Epoch 3/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 23s 26ms/step - AUC: 0.7804 - categorical_accuracy: 0.6996 - loss: 0.5590
Epoch 4/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 23s 26ms/step - AUC: 0.8008 - categorical_accuracy: 0.7165 - loss: 0.5348
Epoch 5/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 42s 26ms/step - AUC: 0.8185 - categorical_accuracy: 0.7369 - loss: 0.5190
Epoch 6/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 23s 26ms/step - AUC: 0.8142 - categorical_accuracy: 0.7330 - loss: 0.5224
Epoch 7/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 24s 27ms/step - AUC: 0.8391 - categorical_accuracy: 0.7509 - loss: 0.4899
Epoch 8/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 24s 27ms/step - AUC: 0.8353 - categorical_accuracy: 0.7504 - loss: 0.4949
Epoch 9/50
891/891 ━━━━━━━━━━━━━━━━━━━━ 23s 26ms/step - AUC: 0.8474 - categorical_accura

In [ ]:
from keras import models
from tensorflow.keras.optimizers import Adam, AdamW

teacher_model1 = models.load_model(path_model +"model_teacher21.h5")
teacher_model2 = models.load_model(path_model +"model_teacher22.h5")
# Compile Distiller
distiller = Distiller(teacher1=teacher_model1, teacher2=teacher_model2, student=student, alpha=0.5, temperature=10)
distiller.compile(
    metrics=['categorical_accuracy','AUC'],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.KLDivergence()
)
distiller.fit(Xtrain, ytrain,
              epochs=50,
              batch_size =16,
              )

In [ ]:
import os
student.save(path_model + "KD2_Succi.h5") # Save model KD2

In [ ]:
pip install openpyxl

In [ ]:
import numpy as np
import json
import os  # Import os to check if the path exists
import pandas as pd  # Import pandas for easy CSV saving
from sklearn.metrics import confusion_matrix  # Import confusion_matrix
# Generate final predictions using logits from both students
ypred = student.predict(Xtest_student)

# Ensure predictions are in the correct format
ypred = np.argmax(ypred, axis=1)  # Getting the predicted class
ytest_true = np.argmax(ytest_student, axis=1)  # Getting the true class

# Compute the confusion matrix
confusion_mat = confusion_matrix(ytest_true, ypred)  # Rename to avoid overwriting

# Calculate TP, TN, FP, FN
TN = confusion_mat[0, 0]  # True Negative
FP = confusion_mat[0, 1]  # False Positive
FN = confusion_mat[1, 0]  # False Negative
TP = confusion_mat[1, 1]  # True Positive

# Create a DataFrame to hold TP, TN, FP, FN values
data = {'TP': [TP],
        'FP': [FP],
        'TN': [TN],
        'FN': [FN]}

df = pd.DataFrame(data)

# Define the path where you want to save the output file
# Make sure this points to a valid directory
output_file = 'Generic_KD2_Succi.xlsx'

# Write the DataFrame to an Excel file
df.to_excel(os.path.join(path_result, output_file), index=False)

print(f"Các giá trị TP, TN, FP, FN đã được ghi vào file {output_file} dưới dạng cột.")